### Probabilistic classification: RF & adjust tuning step


Thresholding in the context of:

1) Compute the difference between p(class) - t(class) **

1) Sorted in descending order

2) Compare max to threshold, if less then we move on to compare the next most likely class against the threshold

3) Continue until probability is equal or greater than threshold, assign that class

4) If no classes get assigned, then that cell is labelled as 'ambiguous

In [78]:
## Importing libraries 

import pandas as pd
import numpy as np 
import random 

from sklearn import preprocessing 
from statistics import mean

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA 

from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedRandomForestClassifier


from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 
from pprint import pprint 

from sklearn.preprocessing import normalize

import matplotlib.pyplot as plt 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve

#import ml_insights as mli 

### Importing data 

Require: 
1. input_files.txt - to contian filenames I want to use. ** currently .csv files

In [79]:
### 1) Importing annotated cells 
#Variables: mylist, inputs 

## obtaining list of files 
with open("D:/Tanrada_classification/imbalance_cortical_training/occipital/annotated/training_slides.txt") as f: 
    mylist= f.read().splitlines()
    
print("Read in: ",len(mylist),"files")

## 2) reading in all those files 
inputs = [] 
for i in mylist: 
    dat = pd.read_csv('D:/Tanrada_classification/imbalance_cortical_training/occipital/annotated/'+i,sep=",")
    ## Changing column names - since these names tend to be inconsistent causing problems 
    dat.columns.values[5] = 'Centroid_X'
    dat.columns.values[6] = 'Centroid_Y'
    dat.columns.values[8] = 'Nucleus: Area ¬µm^2'
    dat.columns.values[9] = 'Nucleus: Length ¬µm'
    dat.columns.values[12] = 'Nucleus: Max diameter ¬µm'
    dat.columns.values[13] = 'Nucleus: Min diameter ¬µm'
    dat.columns.values[14] = 'Cell: Area ¬µm^2'
    dat.columns.values[15] = 'Cell: Length ¬µm'
    dat.columns.values[18] = 'Cell: Max diameter ¬µm'
    dat.columns.values[19] = 'Cell: Min diameter ¬µm'
    #dat_cleaned = dat.iloc[:,0:61] ## SELECTING ONLY RELELVANT COLUMNS 
    print(i," number of features: ", dat.shape[1])
    inputs.append(dat)

print("Extracted:", len(inputs),"files")
print("Note: inconsistencies come from tau necrosis - will be removed later")
#Example
inputs[1].head()


Read in:  8 files
721703_cell_annotations.csv  number of features:  62
721770_cell_annotations.csv  number of features:  61
747377_cell_annotations.csv  number of features:  342
747813_cell_annotations.csv  number of features:  342
747847_cell_annotations.csv  number of features:  342
747848_cell_annotations.csv  number of features:  342
755504_cell_annotations.csv  number of features:  61
755508_cell_annotations.csv  number of features:  61
Extracted: 8 files
Note: inconsistencies come from tau necrosis - will be removed later


,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Detection probability,Nucleus: Area ¬µm^2,Nucleus: Length ¬µm,...,DAB: Membrane: Mean,DAB: Membrane: Median,DAB: Membrane: Min,DAB: Membrane: Max,DAB: Membrane: Std.Dev.,DAB: Cell: Mean,DAB: Cell: Median,DAB: Cell: Min,DAB: Cell: Max,DAB: Cell: Std.Dev.
0,721770.svs,Grey_matter,Unlabelled,PathAnnotationObject,Polygon,6724.3,1282.3,0.8278,16.3984,15.1591,...,0.0376,0.0300,-0.0285,0.2166,0.0383,0.0616,0.0324,-0.0989,0.5054,0.0768
1,721770.svs,Grey_matter,Unlabelled,PathAnnotationObject,Polygon,6667.9,1293.1,0.7811,22.6658,17.4292,...,0.0484,0.0304,-0.0270,0.2093,0.0482,0.0646,0.0502,-0.1584,0.3228,0.0645
2,721770.svs,Grey_matter,Unlabelled,PathAnnotationObject,Polygon,6696.7,1293.7,0.8021,36.8686,21.9999,...,0.0545,0.0429,-0.0138,0.2803,0.0494,0.0643,0.0358,-0.1187,0.4674,0.0766
3,721770.svs,Grey_matter,Unlabelled,PathAnnotationObject,Polygon,6732.0,1295.1,0.8091,17.6270,15.9432,...,0.0422,0.0273,-0.0358,0.2388,0.0472,0.0649,0.0459,-0.1247,0.3948,0.0627
4,721770.svs,Grey_matter,Unlabelled,PathAnnotationObject,Polygon,6928.0,1302.2,0.5366,19.3119,17.7394,...,0.0547,0.0460,-0.0114,0.1927,0.0428,0.0504,0.0419,-0.0355,0.2131,0.0401


In [80]:
# 2.5) Importing in neghbouring cells info (numbers)

nb_mylist = [i[0:6]+'_all_neighbours.csv' for i in mylist]
print("Read in:",len(nb_mylist)," NUMBER OF neighbouring cells files")

# reading in all those files 
nb_inputs = [] 
for i in nb_mylist: 
    dat = pd.read_csv("D:/number_of_neighbours/"+i,sep=",")
    nb_inputs.append(dat)
    
print("Extracted:", len(nb_inputs),"files")

Read in: 8  NUMBER OF neighbouring cells files
Extracted: 8 files


In [81]:
# Inspecting number of columns in NNB files: 
for i in nb_inputs:
    print("Number of features ",i.shape[1])

Number of features  18
Number of features  18
Number of features  18
Number of features  18
Number of features  18
Number of features  18
Number of features  18
Number of features  18


In [82]:
## Extracting only annotated cells from nb files 
inputs_=[]
n=len(inputs)
for i in range(0,n):
    #all cells on slide
    nb_dat_ = nb_inputs[i]
    nb_dat_ = nb_dat_.rename(columns={'X':'Centroid_X','Y':'Centroid_Y'})
    nb_dat = nb_dat_[['Centroid_X','Centroid_Y','NN_10_um','NN_20_um','NN_30_um','NN_40_um','NN_50_um'
                    ,'NN_60_um','NN_70_um','NN_80_um','NN_90_um','NN_100_um']] #so we have removed 'slice_id' or 'Image'
    print("nb_dat shape:", nb_dat.shape)
    #annotated cells with no nb info
    dat = inputs[i]
    print("dat shape:", dat.shape)
    
    #annotated cells with nb info: intersect between 2 dataframes 
    combined = dat.merge(nb_dat,on=['Centroid_X','Centroid_Y'],how='inner',validate='1:1') 
    inputs_.append(combined)
    print("Expected shape:", dat.shape[0],dat.shape[1]+nb_dat.shape[1]-2," Resulting shape:",combined.shape)
    print("--------------------------")
    
print("Succesfully combined nb cell counts to main data")

nb_dat shape: (424899, 12)
dat shape: (424899, 62)
Expected shape: 424899 72  Resulting shape: (424899, 72)
--------------------------
nb_dat shape: (208830, 12)
dat shape: (208830, 61)
Expected shape: 208830 71  Resulting shape: (208830, 71)
--------------------------
nb_dat shape: (232377, 12)
dat shape: (174, 342)
Expected shape: 174 352  Resulting shape: (174, 352)
--------------------------
nb_dat shape: (544006, 12)
dat shape: (159, 342)
Expected shape: 159 352  Resulting shape: (159, 352)
--------------------------
nb_dat shape: (841022, 12)
dat shape: (176, 342)
Expected shape: 176 352  Resulting shape: (176, 352)
--------------------------
nb_dat shape: (343097, 12)
dat shape: (178, 342)
Expected shape: 178 352  Resulting shape: (178, 352)
--------------------------
nb_dat shape: (231371, 12)
dat shape: (231371, 61)
Expected shape: 231371 71  Resulting shape: (231371, 71)
--------------------------
nb_dat shape: (176168, 12)
dat shape: (176168, 61)
Expected shape: 176168 71  R

In [83]:
### 2) Importing hema nucleus mean of all detected cells & location coordinates 
# Variables: hema_mylist, hema_inputs 

hema_mylist = [i[0:6]+'_hema.csv' for i in mylist]    
print("Read in:",len(hema_mylist),"hema files")    


## 4) reading in all those files 
hema_inputs = [] 
for i in hema_mylist: 
    dat = pd.read_csv('D:/Tanrada_classification/hema_novel/'+i,sep=",")
    dat.columns.values[0] = 'Centroid_X' # To fix naming inconsistency problem 
    dat.columns.values[1] = 'Centroid_Y'
    hema_inputs.append(dat)

print("Extracted:",len(hema_inputs),"hema files")  

#Example 
hema_inputs[2].head()


Read in: 8 hema files
Extracted: 8 hema files


,Centroid_X,Centroid_Y,Hematoxylin: Nucleus: Mean
0,4029.3,1035.7,0.4228
1,4035.5,1037.4,0.6438
2,4046.0,1039.4,0.6428
3,4132.2,1039.4,0.6158
4,3883.3,1040.0,0.6310


In [84]:
# Checking if filenames & order of them from mylist, nb_mylist & hema_mylist match
x_nb = [i[0:6] for i in nb_mylist]
x = [i[0:6] for i in mylist]
x_h = [i[0:6] for i in hema_mylist]
print("mylist & nb_list matched?:", x==x_nb)
print("mylist & hema_list matched?:",x==x_h)

mylist & nb_list matched?: True
mylist & hema_list matched?: True


### Normalising hematoxlyin per brain side & discard top 1%

In [85]:
### 1) Get instances needed to be remove for each slide
#Variables: hema_to_remove, hema_inputs 
hema_to_remove = [] 
for h in hema_inputs: 
    h2 = h.copy() 
    hema = h2['Hematoxylin: Nucleus: Mean']
    threshold = hema.quantile(0.99)
    hema_norm = hema/threshold 
    h2['Hematoxylin: Nucleus: Mean'] = hema_norm 
    h2 = h2[h2['Hematoxylin: Nucleus: Mean']>1] # to select instances need removing (keep <=1)
    hema_to_remove.append(h2)

for i in range(0,len(hema_to_remove)): 
    print(i, " No. of cells with normalised Hema >1:",len(hema_to_remove[i]), "from", len(hema_inputs[i]),"detected cells")

0  No. of cells with normalised Hema >1: 4245 from 424899 detected cells
1  No. of cells with normalised Hema >1: 2089 from 208830 detected cells
2  No. of cells with normalised Hema >1: 2324 from 232377 detected cells
3  No. of cells with normalised Hema >1: 5440 from 544006 detected cells
4  No. of cells with normalised Hema >1: 8388 from 841022 detected cells
5  No. of cells with normalised Hema >1: 3431 from 343097 detected cells
6  No. of cells with normalised Hema >1: 2311 from 231371 detected cells
7  No. of cells with normalised Hema >1: 1758 from 176168 detected cells


In [86]:
## 2) Discarding annotated cells if they fit the criteria above 
#Variables: cleaned_inputs, removed  

cleaned_inputs = []
removed = [] 
for n in range(0,(len(inputs_))): #looping through annotated % hema (detected) slides 
    
    i = inputs_[n] #annotated cells 
    h = hema_to_remove[n] #cells we need to remove, may or may not contain annotated cells 
    
    #Find cells that exist in both 'i' & 'h' = cells we want to remove 
    to_remove = i.merge(h,on=['Centroid_X','Centroid_Y'],how='inner',validate='1:1')
    
    #Find cells that only exist in 'i' but not in 'h' = cells we want to retain 
    to_retain = i.merge(h,on=['Centroid_X','Centroid_Y'],how='left',indicator=True,validate='1:1')
    
    #Extract cells we want to retain 
    retained = i[to_retain['_merge']=='left_only']
    
    cleaned_inputs.append(retained)
    removed.append(to_remove)
    print(mylist[n],":",i.shape[0]-retained.shape[0],"cells removed")


721703_cell_annotations.csv : 4245 cells removed
721770_cell_annotations.csv : 2089 cells removed
747377_cell_annotations.csv : 0 cells removed
747813_cell_annotations.csv : 2 cells removed
747847_cell_annotations.csv : 7 cells removed
747848_cell_annotations.csv : 0 cells removed
755504_cell_annotations.csv : 2311 cells removed
755508_cell_annotations.csv : 1758 cells removed


### Checking features

In [87]:
print(cleaned_inputs[0].shape)
list(cleaned_inputs[0].columns)

(420654, 72)


['Image',
 'Name',
 'Class',
 'Parent',
 'ROI',
 'Centroid_X',
 'Centroid_Y',
 'Detection probability',
 'Nucleus: Area ¬µm^2',
 'Nucleus: Length ¬µm',
 'Nucleus: Circularity',
 'Nucleus: Solidity',
 'Nucleus: Max diameter ¬µm',
 'Nucleus: Min diameter ¬µm',
 'Cell: Area ¬µm^2',
 'Cell: Length ¬µm',
 'Cell: Circularity',
 'Cell: Solidity',
 'Cell: Max diameter ¬µm',
 'Cell: Min diameter ¬µm',
 'Nucleus/Cell area ratio',
 'Hematoxylin: Nucleus: Mean',
 'Hematoxylin: Nucleus: Median',
 'Hematoxylin: Nucleus: Min',
 'Hematoxylin: Nucleus: Max',
 'Hematoxylin: Nucleus: Std.Dev.',
 'Hematoxylin: Cytoplasm: Mean',
 'Hematoxylin: Cytoplasm: Median',
 'Hematoxylin: Cytoplasm: Min',
 'Hematoxylin: Cytoplasm: Max',
 'Hematoxylin: Cytoplasm: Std.Dev.',
 'Hematoxylin: Membrane: Mean',
 'Hematoxylin: Membrane: Median',
 'Hematoxylin: Membrane: Min',
 'Hematoxylin: Membrane: Max',
 'Hematoxylin: Membrane: Std.Dev.',
 'Hematoxylin: Cell: Mean',
 'Hematoxylin: Cell: Median',
 'Hematoxylin: Cell: Min',

### Removing DAB & tau necrosis & Image_name

In [88]:
## Removing DAB & tau necrosis ** making sure same dimension
cleaned_inputs_ = []
for i in cleaned_inputs:
    # To remove all DAB features
    to_drop1 = list(i.filter(regex='DAB'))
    dat1 = i[i.columns.drop(to_drop1)]
    # To remove tau necrosis features
    to_drop2 = list(dat1.filter(regex='tau'))
    dat2= dat1[dat1.columns.drop(to_drop2)]
    # To remove Smoothed ****
    to_drop3 = list(dat2.filter(regex='Smoothed'))
    dat3= dat2[dat2.columns.drop(to_drop3)]
    #Remove Image_name
    if ('image_name' in list(dat3.columns)):
        dat =dat3.drop(columns=['Image_name'])
    else:
        dat=dat3
    
    cleaned_inputs_.append(dat)
    print("Initial n_features:",i.shape[1], ", After removal:", dat.shape[1])

Initial n_features: 72 , After removal: 51
Initial n_features: 71 , After removal: 51
Initial n_features: 352 , After removal: 51
Initial n_features: 352 , After removal: 51
Initial n_features: 352 , After removal: 51
Initial n_features: 352 , After removal: 51
Initial n_features: 71 , After removal: 51
Initial n_features: 71 , After removal: 51


### Putting the slides together 

In [89]:
##Variables: labelled_orig, labelled_data 
#1) Put the slides together

labelled_orig = pd.concat(cleaned_inputs_)
print(labelled_orig.shape)

# 2) Extract relevant columns 
dat = labelled_orig.drop(columns=['Name','Parent','ROI']) 
dat.head()


(1031543, 51)


,Image,Class,Centroid_X,Centroid_Y,Detection probability,Nucleus: Area ¬µm^2,Nucleus: Length ¬µm,Nucleus: Circularity,Nucleus: Solidity,Nucleus: Max diameter ¬µm,...,NN_10_um,NN_20_um,NN_30_um,NN_40_um,NN_50_um,NN_60_um,NN_70_um,NN_80_um,NN_90_um,NN_100_um
0,721703.svs,Unlabelled,23317.2,303.63,0.8626,24.0212,17.8053,0.9522,1.0000,6.4203,...,0,1,2,3,3,8,8,9,12,17
1,721703.svs,Unlabelled,23117.0,305.14,0.6539,28.8830,22.6611,0.7068,0.9962,8.9267,...,0,2,4,6,9,12,15,18,25,30
2,721703.svs,Unlabelled,23301.3,310.54,0.8008,57.8022,28.1008,0.9199,1.0000,10.7756,...,0,1,2,5,8,9,10,14,17,20
3,721703.svs,Unlabelled,23204.2,309.59,0.8541,29.3505,21.2158,0.8194,0.9948,7.9087,...,0,0,2,5,7,10,13,21,30,34
4,721703.svs,Unlabelled,23098.8,310.98,0.7954,25.5138,19.5136,0.8420,1.0000,7.4015,...,0,3,5,6,8,10,14,19,24,31


In [90]:
list(dat.columns)

['Image',
 'Class',
 'Centroid_X',
 'Centroid_Y',
 'Detection probability',
 'Nucleus: Area ¬µm^2',
 'Nucleus: Length ¬µm',
 'Nucleus: Circularity',
 'Nucleus: Solidity',
 'Nucleus: Max diameter ¬µm',
 'Nucleus: Min diameter ¬µm',
 'Cell: Area ¬µm^2',
 'Cell: Length ¬µm',
 'Cell: Circularity',
 'Cell: Solidity',
 'Cell: Max diameter ¬µm',
 'Cell: Min diameter ¬µm',
 'Nucleus/Cell area ratio',
 'Hematoxylin: Nucleus: Mean',
 'Hematoxylin: Nucleus: Median',
 'Hematoxylin: Nucleus: Min',
 'Hematoxylin: Nucleus: Max',
 'Hematoxylin: Nucleus: Std.Dev.',
 'Hematoxylin: Cytoplasm: Mean',
 'Hematoxylin: Cytoplasm: Median',
 'Hematoxylin: Cytoplasm: Min',
 'Hematoxylin: Cytoplasm: Max',
 'Hematoxylin: Cytoplasm: Std.Dev.',
 'Hematoxylin: Membrane: Mean',
 'Hematoxylin: Membrane: Median',
 'Hematoxylin: Membrane: Min',
 'Hematoxylin: Membrane: Max',
 'Hematoxylin: Membrane: Std.Dev.',
 'Hematoxylin: Cell: Mean',
 'Hematoxylin: Cell: Median',
 'Hematoxylin: Cell: Min',
 'Hematoxylin: Cell: Max',


### Extracting relevant cell classes

In [91]:
# 1) Check no. of cells / class of our data
print("Total",sum(dat['Class'].value_counts()),"cells")
dat['Class'].value_counts()


Total 1031543 cells


Unlabelled    1029835
Neuron            476
Oligo             445
Ignore            244
Astro             220
Epithelial        159
Endo              123
Ambiguous          33
Fragmented          5
Region              2
fragmented          1
Name: Class, dtype: int64

***Note that we did not use 'ambiguous cells'***

In [92]:
# 2) Selecting only relevant cell classes (Using stardist_error instead of ignore_new)
orig = dat.copy()
dat__ = dat[(dat['Class'] == 'Oligo') | (dat['Class'] == 'Neuron')
          | (dat['Class'] == 'Astro')| (dat['Class'] == 'Epithelial')
          | (dat['Class'] == 'Ignore')| (dat['Class'] == 'Fragmented')| (dat['Class'] == 'Endo')]
dat__=dat__.reset_index(drop=True)

In [93]:
# 3) Changing Epithelial to Endothelial
classes = dat__['Class']
formatted_classes = ['Endo'  if (i == 'Epithelial') else i for i in classes]
dat_ = dat__.copy()
dat_['Class']=formatted_classes

In [94]:
# 4) Checking results from 3)
dat_['Class'].value_counts()

Neuron        476
Oligo         445
Endo          282
Ignore        244
Astro         220
Fragmented      5
Name: Class, dtype: int64

In [95]:
# 6) Group Ignore, Endo & Fragmented cells as a single class called 'Others'
class_ = dat_['Class']
y = ['Others'if i == 'Endo' or i == 'Ignore' or i == 'Fragmented' else i for i in class_ ]
dat = dat_
dat['Class'] = y 
print(dat['Class'].value_counts().sum())
dat['Class'].value_counts()


1672


Others    531
Neuron    476
Oligo     445
Astro     220
Name: Class, dtype: int64

1765
Others    572
Oligo     489
Neuron    463
Astro     241
Name: Class, dtype: int64

In [96]:
#if cv=10
dat['Class'].value_counts()/10

Others    53.1
Neuron    47.6
Oligo     44.5
Astro     22.0
Name: Class, dtype: float64

# Training the model

### Checking for any NA in the data

In [97]:
#checking for NAN 
## NEW 
print("Any NA in the data?: ",dat.isnull().sum().sum()==1)

#dat = dat.dropna()
# dat.isnull().sum().sum()
#print("Any NA in the data?: ",dat.isnull().sum().sum()==1)

Any NA in the data?:  False


### Create train, test sets 

In [98]:
#We are using the entire dataset to train the model, test data will be provided later by Sanne 
X_train_l = dat.drop(columns=['Class']) #,'Image_name','Image_x','Image_y'
X_train = X_train_l.drop(columns=['Image','Centroid_X','Centroid_Y'])
print('training data shape:',X_train_l.shape)
y_train = dat['Class']

training data shape: (1672, 47)


In [99]:
X_train.head()

,Detection probability,Nucleus: Area ¬µm^2,Nucleus: Length ¬µm,Nucleus: Circularity,Nucleus: Solidity,Nucleus: Max diameter ¬µm,Nucleus: Min diameter ¬µm,Cell: Area ¬µm^2,Cell: Length ¬µm,Cell: Circularity,...,NN_10_um,NN_20_um,NN_30_um,NN_40_um,NN_50_um,NN_60_um,NN_70_um,NN_80_um,NN_90_um,NN_100_um
0,0.8900,19.8128,15.8949,0.9855,1.0000,5.2491,4.8900,175.3026,47.1113,0.9925,...,0,0,0,1,5,8,12,16,22,23
1,0.7042,16.1802,16.5761,0.7400,0.9797,6.9006,2.8992,129.4477,44.7169,0.8135,...,0,0,0,2,3,6,8,10,17,21
2,0.9050,40.6065,23.0532,0.9602,1.0000,8.2325,6.4008,170.7491,49.0544,0.8917,...,0,2,2,3,5,5,12,15,17,20
3,0.8948,13.7914,13.2936,0.9807,1.0000,4.3656,4.0313,66.5980,30.8291,0.8805,...,1,2,3,4,4,7,11,15,17,21
4,0.8426,31.2085,20.1525,0.9657,1.0000,7.0218,5.7947,191.4056,50.2637,0.9520,...,0,1,1,4,4,11,12,15,18,23


In [100]:
X_train.columns

Index(['Detection probability', 'Nucleus: Area ¬µm^2', 'Nucleus: Length ¬µm',
       'Nucleus: Circularity', 'Nucleus: Solidity',
       'Nucleus: Max diameter ¬µm', 'Nucleus: Min diameter ¬µm',
       'Cell: Area ¬µm^2', 'Cell: Length ¬µm', 'Cell: Circularity',
       'Cell: Solidity', 'Cell: Max diameter ¬µm', 'Cell: Min diameter ¬µm',
       'Nucleus/Cell area ratio', 'Hematoxylin: Nucleus: Mean',
       'Hematoxylin: Nucleus: Median', 'Hematoxylin: Nucleus: Min',
       'Hematoxylin: Nucleus: Max', 'Hematoxylin: Nucleus: Std.Dev.',
       'Hematoxylin: Cytoplasm: Mean', 'Hematoxylin: Cytoplasm: Median',
       'Hematoxylin: Cytoplasm: Min', 'Hematoxylin: Cytoplasm: Max',
       'Hematoxylin: Cytoplasm: Std.Dev.', 'Hematoxylin: Membrane: Mean',
       'Hematoxylin: Membrane: Median', 'Hematoxylin: Membrane: Min',
       'Hematoxylin: Membrane: Max', 'Hematoxylin: Membrane: Std.Dev.',
       'Hematoxylin: Cell: Mean', 'Hematoxylin: Cell: Median',
       'Hematoxylin: Cell: Min', 'Hem

### My own functions 

In [101]:
## Functions for custom classification metrics 

## Accuracy per class 
def astro_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[0] #astrocytes acc

def neuron_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[1] #neuron acc

def oligo_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[2] #oligo acc

def others_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[3] #ignore acc



## Confusion per class: 

## Astro
def A_as_N(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[0][1] # percentage that A is wrongly classified as N   

def A_as_O(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[0][2] # percentage that A is wrongly classified as O       


def A_as_Others(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[0][3]   

##Neurons 

def N_as_A(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[1][0] 

def N_as_O(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[1][2] 

def N_as_Others(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[1][3] 


## Oligo 

def O_as_A(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[2][0] 

def O_as_N(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[2][1] 

def O_as_Others(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[2][3] 

## Others 

def Others_as_A(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[3][0] 

def Others_as_N(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[3][1] 

def Others_as_O(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[3][2] 


In [102]:
## Functions for custom classification metrics: RAW VALUES 


## Confusion per class: 

## Astro
def A_as_N_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[0][1] # percentage that A is wrongly classified as N   

def A_as_O_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[0][2] # percentage that A is wrongly classified as O       


def A_as_Others_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[0][3]   

##Neurons 

def N_as_A_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[1][0] 

def N_as_O_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[1][2] 

def N_as_Others_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[1][3] 


## Oligo 

def O_as_A_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[2][0] 

def O_as_N_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[2][1] 

def O_as_Others_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[2][3] 

## Others 

def Others_as_A_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[3][0] 

def Others_as_N_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[3][1] 

def Others_as_O_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[3][2] 


In [103]:
### Precision-recall score
def precision_recall_auc(clf,X,y):
    #Variables
    pr_score={}
    
    #get y prob predictions
    y_prob_pred = clf.predict_proba(X)
    
    #Convert true y name into numerical classes
    y_true_numeric = name_to_numeric_classes(y)
    
    #get number of classes
    n_class = list(set(y))
    
    #create PR curve using OVR approach 
    for i in range(len(n_class)): # for each class, calculate roc_curve 
        p, r, thresh = precision_recall_curve(y_true_numeric, y_prob_pred[:,i], pos_label=i)
        pr_score[i] = auc(r,p) #recall on x axis, precision on y axis
        
    #Combine all pr-scores using 'macro' method
    pr_auc = mean(pr_score.values())
    return pr_auc
        

In [104]:
# ADDITIONAL FUNCTIONS

#FUNCTIONS

# Get all class-specific thresholds from best params

def get_threshold(best_params):
    class_thresholds=[]
    for i in best_params:
        t = best_params[i][0]
        class_thresholds.append(t)
    return class_thresholds 

#Thresholding method 5: using ratio, ambiguous cells are: 
# 1) When predicted probabilities < thresholds, i.e. diff = -ve (lower end)
# 2) When more than 1 class-specific threshold is passed, i.e. more than 1 positive diff scores (upper end)

def threshold_list_of_classes_5(y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell (containing 4 class probabilities)
        
        #Get cell 4 class-specific threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference for each of the 4 classes 
        differences = (i-thresholds)/thresholds
        
        #Count number of positive or equal (0) differences 
        count = np.count_nonzero(differences>=0)
        
        if (count==1): #only assign class when 1 class passes the threshold 
            pred_class = np.argmax(differences)
        else: #Otherwise, label as ambiguous (when more than 1 class passes, or when no class passes)
            pred_class=4

        #putting prediction in a list
        thresholded_classes.append(pred_class)

    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_
             
#Thresholding method 4  using ratio instead of raw difference
def threshold_list_of_classes_4 (y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell (containing 4 class probabilities)
        
        #Get cell 4 class-specific threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference for each of the 4 classes 
        differences = (i-thresholds)/thresholds
        
        #Check if there is at last one positive diff probability
        if (np.any(differences>0)): # If true, 
            
            #get index (indicative of class) of the highest probability difference
            pred_class = np.argmax(differences) 
            
        else: #If all diff probabilities are NEGATIVE
            pred_class = 4 # Assign cell class as 'ambiguous'

        #putting prediction in a list
        thresholded_classes.append(pred_class)

    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_


#Thresholding method 3)

def threshold_list_of_classes_3 (y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell (containing 4 class probabilities)
        
        #Get cell 4 class-specific threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference for each of the 4 classes 
        differences = i-thresholds
        
        #Check if there is at last one positive diff probability
        if (np.any(differences>0)): # If true, 
            
            #get index (indicative of class) of the highest probability difference
            pred_class = np.argmax(differences) 
            
        else: #If all diff probabilities are NEGATIVE
            pred_class = 4 # Assign cell class as 'ambiguous'

        #putting prediction in a list
        thresholded_classes.append(pred_class)

    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_

#Thresholding method 2)
def threshold_list_of_classes_2(y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell 
        
        #Get threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference 
        differences = i-thresholds
               
        #get index (indicative of class) of the highest probability
        pred_class = np.argmax(differences)         

        #putting prediction in a list
        thresholded_classes.append(pred_class)

        
    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_


# To convert numeric classes to its corresponding name classes 
def numeric_to_name_classes(numeric_classes):
    output=[]
    for i in numeric_classes:
        if (i==0):
            c = 'Astro'
        elif(i==1):
            c='Neuron'
        elif(i==2):
            c='Oligo'
        elif(i==3):
            c='Others'
        elif(i==4):
            c='Ambiguous'
        else:
            print('SOMETHING IS WRONG')
        output.append(c)
    return output

# To convert name classes to its corresponding numeric classes: 
def name_to_numeric_classes(name_classes):
    output=[]
    for i in name_classes: 
        if (i == 'Astro'): 
            x=0
        elif(i == 'Neuron'): 
            x=1
        elif(i == 'Oligo'): 
            x=2
        elif(i=='Others'):
            x=3
        elif(i=='Ambiguous'):
            x=4
        else:
            print('SOMETHING IS WRONG')
            break
        output.append(x)
    return output
    
# Create roc curve for each class in multi-classification problem

def multiclass_roc_curves(n_class,test_y_numeric,predy):
    
    fpr = {}
    tpr = {}
    thresh ={}
    
    #calcualte roc curve locations 
    for i in range(n_class): # for each class, calculate roc_curve 
        fpr[i], tpr[i], thresh[i] = roc_curve(test_y_numeric, predy[:,i], pos_label=i)
    
    return fpr,tpr,thresh

#Find the best position on the roc curve for multi-classification problem
def best_param_gmean(n_class,fpr,tpr,thresh):
    #calculate g-mean for each threshold 
    #gmeans={}
    best_params={}
    class_names=['Astro','Neuron','Oligo','Others']
    for i in range(n_class):
        tpr_ = tpr[i]
        fpr_ = fpr[i]
        gm = np.sqrt(tpr_*(1-fpr_))
        t = thresh[i]
        #gmeans[i]=gm
        ix = np.argmax(gm)
        best_params[i] = (t[ix],gm[ix],fpr_[ix],tpr_[ix])
        #print(gm)
       # print(class_names[i],'Best Threshold=%f, G-Mean=%.3f, fpr=%f, tpr=%f' % (t[ix], gm[ix],fpr_[ix],tpr_[ix]))
        #print('--------------------------------------------------')
    return best_params

def multiclass_PR_curves(n_class,test_y_numeric,predy):
    
    precision = {}
    recall = {}
    thresh ={}
    
    #calcualte roc curve locations 
    for i in range(n_class): # for each class, calculate roc_curve 
        precision[i], recall[i], thresh[i] = precision_recall_curve(test_y_numeric, predy[:,i], pos_label=i)
    
    return precision,recall,thresh

#Find the best position on the roc curve for multi-classification problem
def best_param_f_score(n_class,precision,recall,thresh):
    #calculate g-mean for each threshold 
    #f_scores={}
    best_params={}
    class_names=['Astro','Neuron','Oligo','Others']
    for i in range(n_class):
        p = precision[i]
        r = recall[i]
        nu=(2*p*r)
        de=(p+r) 
        f_score = np.divide(nu,de,out=np.zeros_like(nu),where=de != 0) #(2*p*r)/(p+r)
        t = thresh[i]
        #f_scores[i]=f_score
        ix = np.argmax(f_score)
        best_params[i] = (t[ix],f_score[ix],p[ix],r[ix])
        #print(gm)
       # print(class_names[i],'Best Threshold=%f, G-Mean=%.3f, fpr=%f, tpr=%f' % (t[ix], gm[ix],fpr_[ix],tpr_[ix]))
        #print('--------------------------------------------------')
    return best_params

#Thresholding function
def prob_thresholding(y_pred_prob,y_pred,threshold):
    thresholded_class =[]
    for i in range(0,len(y_pred_prob)):
        if(max(y_pred_prob[i])<threshold):
            c='Ambiguous'
        else:
            c=y_pred[i]
        thresholded_class.append(c)
    return thresholded_class

#Removing ambiguous class from thresholded class & y_predict
def remove_amb_class(t_class,y_test):
    
    #Get indices of instances with no ambiguous label 
    x = pd.Series(t_class)
    y_pred_no_amb = x[x!='Ambiguous']
    y_pred_no_amb_indices = y_pred_no_amb.index
    
    #Extract these instances fom y_pred
    #y_predict_no_amb = y_predict.iloc[pred_no_amb_indices]
    
    #Subset y_test
    y_test_no_amb = y_test.iloc[y_pred_no_amb_indices]
    
    return (y_pred_no_amb,y_test_no_amb)

### Hyperparameter tuning - random forest

In [105]:
pipeline = Pipeline([
    ('normalizer',MinMaxScaler()),
    ('selector',RFE(SVC(kernel='linear'))), 
    ('clf',BalancedRandomForestClassifier())
])
#pipeline.set_params(clf=RandomForestClassifier())
pipeline.steps

[('normalizer', MinMaxScaler()),
 ('selector', RFE(estimator=SVC(kernel='linear'))),
 ('clf', BalancedRandomForestClassifier())]

In [106]:
ccp_alphas = [float(x) for x in np.linspace(start=0, stop=0.03, num=7) ]
ccp_alphas

[0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03]

cv=10

In [107]:
### Hyper parameters to tune

#Number of trees in random forest 
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

#Number of features to consider at every split
max_features = ['auto', 'sqrt']

#Maximum number of levels in tree 
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

#Minimum number of samples required to split an internal node 
min_samples_split = [2, 5, 10]

#Minimum number of samples required at each leaf node 
min_samples_leaf = [1, 2, 4]

#Method for selecting samples for training each tree 
bootstrap = [True, False]

#sampling strategy
sampling_strategy=['auto','all','not majority','majority']

#ccp_alphas 
ccp_alphas = [float(x) for x in np.linspace(start=0, stop=0.03, num=7) ]#[float(x) for x in np.linspace(start=0, stop=1, num=10) ]

## Create the random grid 
random_grid = {'selector__n_features_to_select':[30,32,34,36,38,40],
                'clf__n_estimators': n_estimators,
               'clf__max_features': max_features,
               'clf__max_depth': max_depth,
               'clf__min_samples_split': min_samples_split,
               'clf__min_samples_leaf': min_samples_leaf,
              'clf__bootstrap': bootstrap,
              'clf__random_state':[42],
               'clf__sampling_strategy':sampling_strategy, 
               'clf__ccp_alpha':ccp_alphas
             # 'clf__class_weight':['balanced']
              } # newly added
#pprint(random_grid)

rf_random = RandomizedSearchCV(pipeline,
                             param_distributions=random_grid, 
                             n_iter=100,
                             cv=10,
                             verbose=2,
                            random_state=42,
                            n_jobs=-1,
                              refit='PR_AUC', # use this metric to evaluate performance of parameters 
                      scoring={'PR_AUC':precision_recall_auc,
                          'roc_auc_ovr_weighted':'roc_auc_ovr_weighted',
                            'roc_auc_ovo':'roc_auc_ovo',
                              'balanced_accuracy':'balanced_accuracy',
                               'f1_weighted':'f1_weighted',
                               'Astro_accuracy': astro_acc,
                               'Neuron_accuracy':neuron_acc,
                               'Oligo_accuracy':oligo_acc,
                               'Others_accuracy':others_acc,
                               'A_as_N':A_as_N,
                               'A_as_O':A_as_O,
                               'A_as_Others':A_as_Others,
                               'N_as_A':N_as_A,
                               'N_as_O':N_as_O,
                               'N_as_Others':N_as_Others,
                               'O_as_A':O_as_A,
                               'O_as_N':O_as_N,
                               'O_as_Others':O_as_Others,
                               'Others_as_A':Others_as_A,
                               'Others_as_N':Others_as_N,
                               'Others_as_O':Others_as_O
                              })

rf_random.fit(X_train,y_train)

print(rf_random.best_score_)
print(rf_random.best_params_)


Fitting 10 folds for each of 100 candidates, totalling 1000 fits
0.8226719689223525
{'selector__n_features_to_select': 32, 'clf__sampling_strategy': 'not majority', 'clf__random_state': 42, 'clf__n_estimators': 1800, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 2, 'clf__max_features': 'auto', 'clf__max_depth': 40, 'clf__ccp_alpha': 0.005, 'clf__bootstrap': True}


Fitting 10 folds for each of 100 candidates, totalling 1000 fits
0.8226719689223525
{'selector__n_features_to_select': 32, 'clf__sampling_strategy': 'not majority', 'clf__random_state': 42, 'clf__n_estimators': 1800, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 2, 'clf__max_features': 'auto', 'clf__max_depth': 40, 'clf__ccp_alpha': 0.005, 'clf__bootstrap': True}

In [108]:
# Digging into more details 
print("PR-AUC:",
     rf_random.cv_results_['mean_test_PR_AUC'][rf_random.best_index_]*100)
print("ROC-AUC:",
     rf_random.cv_results_['mean_test_roc_auc_ovr_weighted'][rf_random.best_index_]*100)
print("ROC-AUC:",
     rf_random.cv_results_['mean_test_roc_auc_ovo'][rf_random.best_index_]*100)

print("Balanced accuracy:",
      rf_random.cv_results_['mean_test_balanced_accuracy'][rf_random.best_index_]*100)

print("F1_weighted:",
      rf_random.cv_results_['mean_test_f1_weighted'][rf_random.best_index_]*100)

print("Astrocyte accuracy:",
      rf_random.cv_results_['mean_test_Astro_accuracy'][rf_random.best_index_]*100)

print("Neuron accuracy:",
      rf_random.cv_results_['mean_test_Neuron_accuracy'][rf_random.best_index_]*100)

print("Oligo accuracy:",
      rf_random.cv_results_['mean_test_Oligo_accuracy'][rf_random.best_index_]*100)

print("Others accuracy:",
      rf_random.cv_results_['mean_test_Others_accuracy'][rf_random.best_index_]*100)


print("Classified A as N:",
      rf_random.cv_results_['mean_test_A_as_N'][rf_random.best_index_]*100)

print("Classified A as O:",
      rf_random.cv_results_['mean_test_A_as_O'][rf_random.best_index_]*100)

print("Classified A as Others:",
      rf_random.cv_results_['mean_test_A_as_Others'][rf_random.best_index_]*100)

print("Classified N as A:",
      rf_random.cv_results_['mean_test_N_as_A'][rf_random.best_index_]*100)

print("Classified N as O:",
      rf_random.cv_results_['mean_test_N_as_O'][rf_random.best_index_]*100)

print("Classified N as Others:",
      rf_random.cv_results_['mean_test_N_as_Others'][rf_random.best_index_]*100)

print("Classified O as A:",
      rf_random.cv_results_['mean_test_O_as_A'][rf_random.best_index_]*100)

print("Classified O as N:",
      rf_random.cv_results_['mean_test_O_as_N'][rf_random.best_index_]*100)

print("Classified O as Others:",
      rf_random.cv_results_['mean_test_O_as_Others'][rf_random.best_index_]*100)


print("Classified Others as A:",
      rf_random.cv_results_['mean_test_Others_as_A'][rf_random.best_index_]*100)

print("Classified Others as N:",
      rf_random.cv_results_['mean_test_Others_as_N'][rf_random.best_index_]*100)

print("Classified Others as O:",
      rf_random.cv_results_['mean_test_Others_as_O'][rf_random.best_index_]*100)
                                                       

PR-AUC: 82.26719689223525
ROC-AUC: 94.54720213043537
ROC-AUC: 93.68336041410387
Balanced accuracy: 73.64612432196988
F1_weighted: 75.44660728939479
Astrocyte accuracy: 65.45454545454545
Neuron accuracy: 74.59663120567376
Oligo accuracy: 69.4040404040404
Others accuracy: 85.12928022361986
Classified A as N: 14.545454545454545
Classified A as O: 6.8181818181818175
Classified A as Others: 13.18181818181818
Classified N as A: 13.834219858156027
Classified N as O: 0.42109929078014185
Classified N as Others: 11.14804964539007
Classified O as A: 11.712121212121211
Classified O as N: 0.0
Classified O as Others: 18.883838383838388
Classified Others as A: 3.7700908455625433
Classified Others as N: 5.066387141858839
Classified Others as O: 6.03424178895877


PR-AUC: 82.26719689223525
ROC-AUC: 94.54720213043537
ROC-AUC: 93.68336041410387
Balanced accuracy: 73.64612432196988
F1_weighted: 75.44660728939479
Astrocyte accuracy: 65.45454545454545
Neuron accuracy: 74.59663120567376
Oligo accuracy: 69.4040404040404
Others accuracy: 85.12928022361986
Classified A as N: 14.545454545454545
Classified A as O: 6.8181818181818175
Classified A as Others: 13.18181818181818
Classified N as A: 13.834219858156027
Classified N as O: 0.42109929078014185
Classified N as Others: 11.14804964539007
Classified O as A: 11.712121212121211
Classified O as N: 0.0
Classified O as Others: 18.883838383838388
Classified Others as A: 3.7700908455625433
Classified Others as N: 5.066387141858839
Classified Others as O: 6.03424178895877

## Manual cross validation, using PR curves

In [109]:
### manual cross-validation method 
x= X_train
y=y_train
## Setting up cross-validation 
skf = StratifiedKFold(n_splits=10) # shuffling = False, no need to set random_state
skf.get_n_splits(x,y) # using only training data
print(skf)

#for train_index, test_index in skf.split(x,y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("Train_size:", train_index.size, "Test_size:", test_index.size)
    
## Training 
accuracies = []
accuracies_c = []

t_accuracies= []
t_accuracies_c= []

reports = []
reports_c = []

t_reports= []
t_reports_c= []

confusion_matrices = []
confusion_matrices_c = [] 

t_confusion_matrices=[]
t_confusion_matrices_c=[]

#train_features =[] 
#train_n_features=[]
y_preds = []
y_preds_c = []

y_preds_t =[]
y_preds_t_c =[]

y_prob_preds = []
y_prob_preds_c = []

y_cv_test=[]
x_cv_test=[]

roc_auc_scores=[]
roc_auc_scores_c=[]

log_losses=[]
log_losses_c=[]

#brier_scores=[]
#brier_scores_c=[]

best_parameters=[]
best_parameters_c=[]

for train_index, test_index in skf.split(x,y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("Train_size:", train_index.size, "Test_size:", test_index.size)
    x_train_, x_test_ = x.iloc[train_index], x.iloc[test_index]
    y_train_, y_test_ = y[train_index], y[test_index]
    x_test_l = X_train_l.iloc[test_index]
    #print("n of each cell types at test:\n", y_test_.value_counts())
    
    ## 1) Create classifier 
    
    model=Pipeline([
    ('normalizer',MinMaxScaler()),
    ('selector',RFE(SVC(kernel='linear'),n_features_to_select=32)), 
    ('clf', BalancedRandomForestClassifier(sampling_strategy= 'not majority', random_state= 42,
                                           n_estimators= 1800, min_samples_split= 2, min_samples_leaf= 2,
                                           max_features= 'auto', max_depth= 40, ccp_alpha= 0.005, bootstrap= True))
                                           ### MAKE SURE THESE ARE CORRECT 
                
])
    
# {'selector__n_features_to_select': 32, 'clf__sampling_strategy': 'not majority', 'clf__random_state': 42,
#  'clf__n_estimators': 1800, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 2,
#  'clf__max_features': 'auto', 'clf__max_depth': 40, 'clf__ccp_alpha': 0.005, 'clf__bootstrap': True}

    # 2) Train the calibrated classifier with 'training' data (x,y) 
    model.fit(x_train_,y_train_)
    
    # 3) Get class probability predictions for 'test' data 
    y_prob_predict = model.predict_proba(x_test_)
    
    # 4.1) For thresholding: convert y_test_ from name classes to numeric classes
    y_test_numeric = name_to_numeric_classes(y_test_)
    
    # 4.2) For thresholding: use predicted class probabilities to calculate ROC curve for each class vs rest
    
    precision,recall,thresh = multiclass_PR_curves(4,y_test_numeric,y_prob_predict)
    
    # 4.3) For thresholding: from ROC curves, find the best location (fpr,tpr,thresh) for each class 
    # Evaluated based on g-mean 
    best_params_ = best_param_f_score(4,precision,recall,thresh)
    best_parameters.append(best_params_)
    
    # 4.4) For thresholding: apply thresholding to each class to create crisp class label 
    t_class = threshold_list_of_classes_5(y_prob_predict,best_params_)
    
    # 5) Get class labels using default thresholding value (0.5)
    y_predict = model.predict(x_test_)
    
    # 6) Put predictions (labels &probabilities & t_labels) in the corresponding list
    y_preds.append(y_predict)
    
    y_prob_preds.append(y_prob_predict)
    
    y_preds_t.append(t_class)
    
    y_cv_test.append(y_test_) # for visualisation purposes later on
    x_cv_test.append(x_test_l)
    
    # 7) Remove 'ambiguous class' from t_class & y_test_ - for accuracy calculation
    (y_predict_no_amb,y_test_no_amb) = remove_amb_class(t_class,y_test_)
    
    # 8) Calculate and put Performance metric (balanced accuracy) per fold into a list
    accuracies.append(balanced_accuracy_score(y_test_,y_predict)) ## using BALANCED ACC.
    
    t_accuracies.append(balanced_accuracy_score(y_test_no_amb,y_predict_no_amb))
    
    #8.1) Compute classification reports
    reports.append(classification_report(y_test_,y_predict,output_dict=True)) 
  #  reports_c.append(classification_report(y_test_,y_predict_c,output_dict=True)) 
    
    t_reports.append(classification_report(y_test_no_amb,y_predict_no_amb,output_dict=True))
    
    # 9) Calculate and put ROC AUC scores per fold into a list 
    roc_auc_scores.append(roc_auc_score(y_test_,y_prob_predict,multi_class='ovr',average='weighted'))
    
    #9.1) Calculate and put log loss per fold into a list
    log_losses.append(log_loss(y_test_,y_prob_predict))

    
    # 10) Create confusion matrices for default & thresholded results per fold then put in a list 
    cm = confusion_matrix(y_test_,y_predict, labels=["Astro","Neuron","Oligo","Others"]) #,normalize='true'
    
    cm_t = confusion_matrix(y_test_no_amb,y_predict_no_amb, labels=["Astro","Neuron","Oligo","Others"])#,normalize='true'
    
    confusion_matrices.append(cm)
    
    t_confusion_matrices.append(cm_t)

print('mean ROC AUC:',mean(roc_auc_scores))
print('--------------------------------')
print('mean log loss:',mean(log_losses))
print('--------------------------------')

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
mean ROC AUC: 0.9454720213043535
--------------------------------
mean log loss: 0.6257403525336385
--------------------------------


Extracting information from best parameters

In [110]:
#Thresholds 
astro_t = [] 
neuron_t=[]
oligo_t=[]
others_t=[]
#G-means
astro_gm=[]
neuron_gm=[]
oligo_gm=[]
others_gm=[]

#Info extraction 
for fold in best_parameters:
    a_t = fold[0][0]
    n_t = fold[1][0]
    o_t = fold[2][0]
    ot_t = fold[3][0]
    
    a_gm = fold[0][1]
    n_gm = fold[1][1]
    o_gm = fold[2][1]
    ot_gm = fold[3][1]
    
    astro_t.append(a_t)
    neuron_t.append(n_t)
    oligo_t.append(o_t)
    others_t.append(ot_t)
    
    astro_gm.append(a_gm)
    neuron_gm.append(n_gm)
    oligo_gm.append(o_gm)
    others_gm.append(ot_gm)

In [111]:
print("NON CALIBRATED")
print('mean astro threshold:', mean(astro_t), ', mean f1_macro:',mean(astro_gm))
print('mean neuron threshold:', mean(neuron_t), ', mean f1_macro:',mean(neuron_gm))
print('mean oligo threshold:', mean(oligo_t), ', mean f1_macro:',mean(oligo_gm))
print('mean others threshold:', mean(others_t), ', mean f1_macro:',mean(others_gm))
print(mean(astro_t)+mean(neuron_t)+mean(oligo_t)+mean(others_t)) #is it okay that it is above 1? 

NON CALIBRATED
mean astro threshold: 0.35793807456439586 , mean f1_macro: 0.6340967926090835
mean neuron threshold: 0.2695591565163307 , mean f1_macro: 0.8470746099012754
mean oligo threshold: 0.2639737969421004 , mean f1_macro: 0.8587408110939749
mean others threshold: 0.5144890330406827 , mean f1_macro: 0.8309850166507259
1.4059600610635097


**NO Thresholding:** Non-calibrated

In [112]:
#Confusion matrix across 10 folds, WITHOUT thresholding 
print('with no thresholding:',mean(accuracies)*100)
print('Macro avg F1 ',mean([i['macro avg']['f1-score'] for i in reports])*100)
print('Weighted avg F1 ',mean([i['weighted avg']['f1-score'] for i in reports])*100)

print("--------------------------")
C=sum(confusion_matrices)
final_cm =  C.astype('float') / C.sum(axis=1)[:, np.newaxis]*100 #normalize(sum(confusion_matrices))*100
print(C)
print(final_cm)
print("--------------------------")
print("Astro accuracy",final_cm[0][0])
print("Neuron accuracy",final_cm[1][1])
print("Oligo accuracy",final_cm[2][2])
print("Others accuracy",final_cm[3][3])
print("--------------------------")
# F1-score per class: 
print('Astro f1-score ',mean([i['Astro']['f1-score'] for i in reports])*100)
print('Neuron f1-score ',mean([i['Neuron']['f1-score'] for i in reports])*100)
print('Oligo f1-score ',mean([i['Oligo']['f1-score'] for i in reports])*100)
print('Others f1-score ',mean([i['Others']['f1-score'] for i in reports])*100)
print("--------------------------")
print('Macro avg precision',mean([i['macro avg']['precision'] for i in reports])*100)
print('Macro avg recall ',mean([i['macro avg']['recall'] for i in reports])*100)

with no thresholding: 73.64612432196986
Macro avg F1  72.97174847999646
Weighted avg F1  75.44660728939479
--------------------------
[[144  32  15  29]
 [ 66 355   2  53]
 [ 52   0 309  84]
 [ 20  27  32 452]]
[[65.45454545 14.54545455  6.81818182 13.18181818]
 [13.86554622 74.57983193  0.42016807 11.13445378]
 [11.68539326  0.         69.43820225 18.87640449]
 [ 3.76647834  5.08474576  6.02636535 85.12241055]]
--------------------------
Astro accuracy 65.45454545454545
Neuron accuracy 74.57983193277312
Oligo accuracy 69.43820224719101
Others accuracy 85.12241054613936
--------------------------
Astro f1-score  57.78839338458975
Neuron f1-score  79.43690440854768
Oligo f1-score  75.64718874858124
Others f1-score  79.01450737826718
--------------------------
Macro avg precision 74.86092179569212
Macro avg recall  73.64612432196986


**Thresholding:** Non-calibrated

In [113]:
#0.5 
#Confusion matrix across 10 folds, WITH thresholding 
print('with thresholding (non-calibrated) ACC :',mean(t_accuracies)*100)
print('Macro avg F1 ',mean([i['macro avg']['f1-score'] for i in t_reports])*100)
print('Weighted avg F1 ',mean([i['weighted avg']['f1-score'] for i in t_reports])*100)
print("--------------------------")
C_t=sum(t_confusion_matrices)
final_cm_t =  C_t.astype('float') / C_t.sum(axis=1)[:, np.newaxis]*100
print(C_t)
print(final_cm_t)
print("--------------------------")
print("Astro accuracy",final_cm_t[0][0])
print("Neuron accuracy",final_cm_t[1][1])
print("Oligo accuracy",final_cm_t[2][2])
print("Others accuracy",final_cm_t[3][3])
print('------------------------------')
# F1-score per class: 
print('Astro f1-score ',mean([i['Astro']['f1-score'] for i in t_reports])*100)
print('Astro precision ',mean([i['Astro']['precision'] for i in t_reports])*100)
print('Astro recall ',mean([i['Astro']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Neuron f1-score ',mean([i['Neuron']['f1-score'] for i in t_reports])*100)
print('Neuron precision ',mean([i['Neuron']['precision'] for i in t_reports])*100)
print('Neuron recall ',mean([i['Neuron']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Oligo f1-score ',mean([i['Oligo']['f1-score'] for i in t_reports])*100)
print('Oligo precision ',mean([i['Oligo']['precision'] for i in t_reports])*100)
print('Oligo recall ',mean([i['Oligo']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Others f1-score ',mean([i['Others']['f1-score'] for i in t_reports])*100)
print('Others precision ',mean([i['Others']['precision'] for i in t_reports])*100)
print('Others recall ',mean([i['Others']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Macro avg precision',mean([i['macro avg']['precision'] for i in t_reports])*100)
print('Macro avg recall ',mean([i['macro avg']['recall'] for i in t_reports])*100)

# Checking on disagreements 


thresholded_preds = pd.concat([pd.DataFrame(i) for i in y_preds_t])
thresholded_preds = thresholded_preds.rename(columns={0:'t_Class'})
thresholded_preds = thresholded_preds.reset_index(drop=True)


preds = pd.concat([pd.DataFrame(i) for i in y_preds])
preds = preds.rename(columns={0:'Class'})
preds = preds.reset_index(drop=True)

truth = pd.concat([pd.DataFrame(i) for i in y_cv_test])
truth = truth.rename(columns={'Class':'Truth'})
truth = truth.reset_index(drop=True)

# x_truth = pd.concat([pd.DataFrame(i[['Image','Centroid_X','Centroid_Y']]) for i in x_cv_test])

#Combine absolute prediction to thresholded prediction

#get predicted probabilities
p_probs=pd.concat([pd.DataFrame(i) for i in y_prob_preds])
p_probs= p_probs.rename(columns={0:'Astro',1:'Neuron',2:'Oligo',3:'Others'})
p_probs = p_probs.reset_index(drop=True)

results = thresholded_preds.copy()
results.loc[:,'Class']=preds
results.loc[:,'Truth'] = truth
results.loc[:,'Astro'] = p_probs['Astro']
results.loc[:,'Neuron'] = p_probs['Neuron']
results.loc[:,'Oligo'] = p_probs['Oligo']
results.loc[:,'Others'] = p_probs['Others']
# results.loc[:,'Image'] = x_truth['Image']
# results.loc[:,'Centroid_X'] = x_truth['Centroid_X']
# results.loc[:,'Centroid_Y'] = x_truth['Centroid_Y']


#Calculate agreement between the two 
results.loc[:,'agreement'] = (results['t_Class']==results['Class'])*1
agreements = results['agreement'].value_counts()
print('Agreement: ',agreements[1],'/',agreements[1]+agreements[0],'=> ',(agreements[1]/(agreements[1]+agreements[0])*100,'%') )
print('Disagreement: ',agreements[0],'/',agreements[1]+agreements[0],'=> ',(agreements[0]/(agreements[1]+agreements[0])*100,'%') )
print('------------------------------')
# Of those disagreed, what are they? (those with prob < 0.5)
print('Of the disagreements, what are they?')
disagreed = results[results['agreement']==0]
disagreed['Class'].value_counts()

with thresholding (non-calibrated) ACC : 85.02476036879838
Macro avg F1  84.60261078116721
Weighted avg F1  87.00641848988474
--------------------------
[[107  22   8   9]
 [ 15 363   3  18]
 [ 15   0 341  22]
 [ 13  26  29 397]]
[[73.28767123 15.06849315  5.47945205  6.16438356]
 [ 3.7593985  90.97744361  0.7518797   4.5112782 ]
 [ 3.96825397  0.         90.21164021  5.82010582]
 [ 2.79569892  5.59139785  6.23655914 85.37634409]]
--------------------------
Astro accuracy 73.28767123287672
Neuron accuracy 90.97744360902256
Oligo accuracy 90.21164021164022
Others accuracy 85.3763440860215
------------------------------
Astro f1-score  72.07685138719621
Astro precision  72.45299145299145
Astro recall  73.58063573853048
--------------------------
Neuron f1-score  89.74498348578356
Neuron precision  88.63865705111368
Neuron recall  91.2350232509807
--------------------------
Oligo f1-score  89.53548422699939
Oligo precision  89.5567234612701
Oligo recall  89.81387927036839
----------------

Others    175
Astro     141
Neuron     36
Oligo      28
Name: Class, dtype: int64

In [114]:
thresholded_preds['t_Class'].value_counts()

Others       446
Neuron       411
Oligo        381
Ambiguous    284
Astro        150
Name: t_Class, dtype: int64

In [115]:
results.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement
0,Oligo,Oligo,Oligo,0.053200,0.008566,0.824747,0.113488,1
1,Others,Others,Others,0.029127,0.015659,0.072450,0.882764,1
2,Neuron,Neuron,Neuron,0.353800,0.532260,0.028031,0.085910,1
3,Oligo,Oligo,Oligo,0.023378,0.003473,0.847142,0.126006,1
4,Ambiguous,Astro,Astro,0.664605,0.165337,0.053372,0.116687,0


In [116]:
results[results['t_Class']!=results['Truth']]

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement
4,Ambiguous,Astro,Astro,0.664605,0.165337,0.053372,0.116687,0
5,Ambiguous,Others,Oligo,0.298695,0.038325,0.300075,0.362905,0
7,Oligo,Oligo,Others,0.067006,0.014080,0.703508,0.215405,1
10,Ambiguous,Others,Others,0.057702,0.364026,0.004525,0.573747,0
13,Ambiguous,Astro,Astro,0.661711,0.191389,0.036769,0.110131,0
...,...,...,...,...,...,...,...,...
1655,Ambiguous,Astro,Oligo,0.490975,0.069880,0.092155,0.346990,0
1660,Ambiguous,Others,Neuron,0.238278,0.167863,0.039806,0.554054,0
1662,Ambiguous,Astro,Oligo,0.368320,0.032069,0.261882,0.337729,0
1665,Ambiguous,Astro,Oligo,0.410179,0.044015,0.138716,0.407090,0


In [117]:
x_test =pd.concat(x_cv_test)
x_test=x_test.reset_index(drop=True)
x_test_subset=x_test[['Image','Centroid_X','Centroid_Y']]

In [118]:
results_=results.copy()
results_=results.reset_index(drop=True)

In [119]:
results_ = results_.join(x_test_subset)

In [120]:
results_.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement,Image,Centroid_X,Centroid_Y
0,Oligo,Oligo,Oligo,0.053200,0.008566,0.824747,0.113488,1,721703.svs,10595.8,8928.6
1,Others,Others,Others,0.029127,0.015659,0.072450,0.882764,1,721703.svs,10681.8,8947.5
2,Neuron,Neuron,Neuron,0.353800,0.532260,0.028031,0.085910,1,721703.svs,10713.9,9006.9
3,Oligo,Oligo,Oligo,0.023378,0.003473,0.847142,0.126006,1,721703.svs,10706.4,9014.4
4,Ambiguous,Astro,Astro,0.664605,0.165337,0.053372,0.116687,0,721703.svs,10805.9,9069.3


In [121]:
results_['Image'].value_counts()

755504.svs    320
755508.svs    282
721703.svs    257
747848.svs    174
721770.svs    171
747377.svs    168
747847.svs    158
747813.svs    142
Name: Image, dtype: int64

In [122]:
incorrect = results_[results_['t_Class']!=results_['Truth']]
incorrect.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement,Image,Centroid_X,Centroid_Y
4,Ambiguous,Astro,Astro,0.664605,0.165337,0.053372,0.116687,0,721703.svs,10805.9,9069.3
5,Ambiguous,Others,Oligo,0.298695,0.038325,0.300075,0.362905,0,721703.svs,10809.4,9102.1
7,Oligo,Oligo,Others,0.067006,0.014080,0.703508,0.215405,1,721703.svs,10816.7,9182.5
10,Ambiguous,Others,Others,0.057702,0.364026,0.004525,0.573747,0,721703.svs,10885.0,9224.1
13,Ambiguous,Astro,Astro,0.661711,0.191389,0.036769,0.110131,0,721703.svs,10963.0,9249.8


In [67]:
# incorrect_as_file = incorrect[['Image','Truth','Centroid_X','Centroid_Y','t_Class']]
# path_ = 'D:/Tanrada_classification/imbalance_cortical_training/cortical_full_slide_predictions/Probabilistic_classification/Model8_occipital_classifier/cell_inspection/incorrect.txt'
# incorrect_as_file.to_csv(path_, sep='\t',index=False)

In [123]:
correct = results_[results_['t_Class']==results_['Truth']]
correct.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement,Image,Centroid_X,Centroid_Y
0,Oligo,Oligo,Oligo,0.053200,0.008566,0.824747,0.113488,1,721703.svs,10595.8,8928.6
1,Others,Others,Others,0.029127,0.015659,0.072450,0.882764,1,721703.svs,10681.8,8947.5
2,Neuron,Neuron,Neuron,0.353800,0.532260,0.028031,0.085910,1,721703.svs,10713.9,9006.9
3,Oligo,Oligo,Oligo,0.023378,0.003473,0.847142,0.126006,1,721703.svs,10706.4,9014.4
6,Others,Others,Others,0.084398,0.024933,0.139737,0.750933,1,721703.svs,10864.0,9153.5


In [69]:
# correct_as_file = correct[['Image','Truth','Centroid_X','Centroid_Y','t_Class']]
# path_ = 'D:/Tanrada_classification/imbalance_cortical_training/cortical_full_slide_predictions/Probabilistic_classification/Model8_occipital_classifier/cell_inspection/correct.txt'
# correct_as_file.to_csv(path_, sep='\t',index=False)